In [5]:
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import bqplot as bq
import folium
import branca
import ipywidgets
from ipywidgets import interact
import vincent
import json
#bokeh
from bokeh.plotting import figure, output_file 
from bokeh.io import show,output_notebook
from bokeh.palettes import Plasma
from bokeh.models import ColumnDataSource
from bokeh.transform import factor_cmap
from bokeh.models import FuncTickFormatter
from folium import plugins
from folium.plugins import HeatMap

In [6]:
data = pd.read_csv("201712-citibike-tripdata_cleaned.csv")

In [7]:
start_stations = dict(zip(data["start station name"], data["start station id"]))
start_stations["None"] = None
end_stations = dict(zip(data["end station name"], data["end station id"]))
end_stations["None"]=None
hour = dict(zip(data["hour"]+1,data["hour"]))
map_options = {"Mapbox Bright":"Mapbox Bright","Mapbox Control Room":"Mapbox Control Room","OpenStreetMap":"OpenStreetMap","Mapbox Bright":"Mapbox Bright","Stamen Toner":"Stamen Toner","Stamen Terrain":"Stamen Terrain","CartoDBDark":"CartoDB dark_matter"}

In [8]:
def trip_by_hour(hour_val):
    loc1 = data.groupby("start station id").first()
    location = loc1.loc[:,["start station longitude","start station latitude","start station name"]]
    hour = data[data["hour"]==hour_val]
    departure=hour.groupby("start station id").count().iloc[:,[0]]
    departure.columns=["Counts of Departure"]
    #print(type(departure))
    arrival= hour.groupby("end station id").count().iloc[:,[0]]
    arrival.columns=["Counts of Arrival"]
    #print(type(arrival))
    #joining arrival and departure
    trip = departure.join(location).join(arrival)
    return trip
    #print(departure)

In [9]:
def plot_station_counts(trip_counts):
    # generate a new map
    folium_map = folium.Map(location=[40.738, -73.98],
                            tiles="CartoDB dark_matter",
                            zoom_start=13,
                            width='50%')

    # for each row in the data, add a cicle marker
    for index, row in trip_counts.iterrows():
        # calculate net departures
        net_departures = (row["Counts of Departure"]-row["Counts of Arrival"])
        
        # generate the popup message that is shown on click.
        popup_text = "{}<br> total departures: {}<br> total arrivals: {}<br> net departures: {}"
        popup_text = popup_text.format(row["start station name"],
                          row["Counts of Arrival"],
                          row["Counts of Departure"],
                          net_departures)
        
        # radius of circles
        radius = net_departures/20
        
        # choose the color of the marker
        if net_departures>0:
            #color="#FFCE00" # orange
             color="#007849" # green
            #color="#E37222" # tangerine
        else:
             color="#0375B4" # blue
            # color="#FFCE00" # yellow            
            #color="#0A8A9F" # teal
        
        # add marker to the map
        folium.CircleMarker(location=(row["start station latitude"],
                                      row["start station longitude"]),
                            radius=radius,
                            color=color,
                            popup=popup_text,
                            fill=True).add_to(folium_map)
        #data['points'] = list(zip(data["start station latitude"], data["start station longitude"]))
    #trip_counts['points'] = list(zip(trip_counts["start station latitude"], trip_counts["start station longitude"]))
    #for i in trip_counts["points"]:
    #    folium.Marker(i).add_to(folium_map)
    #folium.PolyLine(trip_counts["points"], color="red", weight=2.5, opacity=0.2).add_to(folium_map)
    return folium_map

In [10]:
trip_counts = trip_by_hour(9)
plot_station_counts(trip_counts)

In [11]:
def plot_station_by_hour(trip_counts,tiles):
    # generate a new map
    folium_map = folium.Map(location=[40.738, -73.98],
                            tiles="CartoDB dark matter",
                            zoom_start=13,
                            width='50%')

    # for each row in the data, add a cicle marker
    for index, row in trip_counts.iterrows():
        # calculate net departures
        net_departures = (row["Counts of Departure"]-row["Counts of Arrival"])
        
        # generate the popup message that is shown on click.
        popup_text = "{}<br> total departures: {}<br> total arrivals: {}<br> net departures: {}"
        popup_text = popup_text.format(row["start station name"],
                          row["Counts of Arrival"],
                          row["Counts of Departure"],
                          net_departures)
        
        # radius of circles
        radius = net_departures/20
        
        # choose the color of the marker
        if net_departures>0:
            #color="#FFCE00" # orange
             color="#007849" # green
            #color="#E37222" # tangerine
        else:
             color="#0375B4" # blue
            # color="#FFCE00" # yellow            
            #color="#0A8A9F" # teal
        
        # add marker to the map
        #folium.CircleMarker(location=(row["start station latitude"],
        #                              row["start station longitude"]),
        #                    radius=radius,
        #                    color=color,
        #                    popup=popup_text,
        #                    fill=True).add_to(folium_map)
        #data['points'] = list(zip(data["start station latitude"], data["start station longitude"]))
    trip_counts['points'] = list(zip(trip_counts["start station latitude"], trip_counts["start station longitude"]))
    for i in trip_counts["points"]:
        folium.Marker(i).add_to(folium_map)
    folium.PolyLine(trip_counts["points"], color="red", weight=2.5, opacity=0.2).add_to(folium_map)
    return folium_map

In [12]:
from bokeh.models import HoverTool

def route(start_station,end_station,fig):
    val = data[(data["start station id"]==start_station) & (data["end station id"]==end_station)]
    bike = val.groupby(["hour"]).count().iloc[:,[11]].reset_index()
    #output_file("line.html")
    #f = branca.element.Figure()
    bike["hour"] = bike["hour"]+1
    scatter_points = {
        'x': bike["hour"],
        'y': bike["bikeid"],
        }
    scatter_chart = vincent.Scatter(scatter_points,
                       iter_idx='x',
                       width=450,
                      height=250)
    scatter_json = scatter_chart.to_json()
    scatter_dict = json.loads(scatter_json)
    #popup = folium.Popup(max_width=650)
    v = folium.Vega(scatter_dict, height='50%', width='25%',left='50%',position='absolute')
    fig.add_child(v)
    return fig

In [13]:
def route_other(start_station,fig):
    val = data[data["start station id"]==start_station]
    bike = val.groupby(["hour"]).count().iloc[:,[11]].reset_index()
    #print(bike)
    bike["hour"] = bike["hour"]+1
    #print(bike)
    #output_file("line.html")
    #f = branca.element.Figure()
    #print(bike)
    scatter_points = {
        'x': bike["hour"],
        'y': bike["bikeid"],
        }
    scatter_chart = vincent.Scatter(scatter_points,
                       iter_idx='x',
                       width=450,
                      height=250)
    scatter_json = scatter_chart.to_json()
    scatter_dict = json.loads(scatter_json)
    #popup = folium.Popup(max_width=650)
    v = folium.Vega(scatter_dict, height='50%', width='25%',left='50%',position='absolute')
    fig.add_child(v)
    return fig

In [14]:
def plot_station(trip_counts,tiles,val):
    # generate a new map
    folium_map = folium.Map(location=[40.738, -73.98],
                            tiles=tiles,
                            zoom_start=10,control_scale=True,left='0%',
                            width='50%',position='absolute',height='50%')
    #print(trip_counts)
    points=[]
    #popup = folium.Popup()
    #bike = val.groupby(["hour"]).count().iloc[:,[0]].reset_index()
    for index, row in trip_counts.iterrows():
        
        #net_departures = (row["Counts of Departure"]-row["Counts of Arrival"])
        #scatter_points = {
        #'x': bike["hour"],
        #'y': bike["bikeid"],
        #'title' : row["start station name"]
        #}
        #scatter_chart = vincent.Scatter(scatter_points,
        #                        iter_idx='x',
        #                        width=600,
        #                        height=300)
        #scatter_json = scatter_chart.to_json()
        #scatter_dict = json.loads(scatter_json)
        #popup = folium.Popup(max_width=650)
        #folium.Vega(scatter_dict, height=350, width=650).add_to(popup)
        
        popup_text = "{}"
        popup_text_end = "{}"
        popup_text = popup_text.format(row["start station name"])
                          #row["Counts of Arrival"],
                          #row["Counts of Departure"],
                          #net_departures)
        
        popup_text_end = popup_text_end.format(row["end station name"])
                          #row["Counts of Arrival"],
                          #row["Counts of Departure"],
                          #net_departures)
        folium.Marker(location=(row["start station latitude"],
                                      row["start station longitude"]),icon=folium.Icon(color='blue',icon='bicycle',prefix='fa'),
                            popup=popup_text).add_to(folium_map)
        folium.Marker(location=(row["end station latitude"],
                                      row["end station longitude"]),icon=folium.Icon(color='green',icon='bicycle',prefix='fa'),
                            popup=popup_text_end).add_to(folium_map)
    #print(trip_counts)    
    points=list(zip(trip_counts["start station latitude"], trip_counts["start station longitude"]))
    #a=len(points)
    val = list(zip(trip_counts["end station latitude"], trip_counts["end station longitude"]))
    try:
           for i in val:
                points.append(i)
    except:
            print("No Route Found!")
    if(len(points)==0):
        print("No Route Found!")
    #for i in points:
    #    folium.Marker(i).add_to(folium_map)
    #points= list(set(points))
    folium.PolyLine(points, color="red", weight=2.5, opacity=1).add_to(folium_map)
    #print("Number of Bike Trips Recorded: " , a)
    return folium_map

In [15]:
def f(start_station,end_station,hour,tiles):
    fig = branca.element.Figure()
    if(start_station==None and end_station==None):
        #hour_val = val[val["hour"]==hour]
        val=trip_by_hour(hour)
        #print(val.head())
        return plot_station_by_hour(val,tiles)
    elif(end_station==None):
        val = data[(data["start station id"]==start_station)]
        val = val.loc[:,["bikeid","start station name" , "end station name", "start station longitude","start station latitude","hour","end station latitude","end station longitude","start station id","end station id"]]
        hour_val = val[val["hour"]==hour]
        route_other(start_station,fig)
        #print(hour_val)
        #return(plot_station(hour_val))
    elif(start_station==None):
        val = data[(data["end station id"]==end_station)]
        val = val.loc[:,["bikeid","start station longitude","start station name" , "end station name","start station latitude","hour","end station latitude","end station longitude","start station id","end station id"]]
        hour_val = val[val["hour"]==hour]
        route_other(end_station,fig)
        #return(plot_station(hour_val))
    else:
        val = data[(data["start station id"]==start_station) & (data["end station id"]==end_station)]
        val = val.loc[:,["bikeid","start station longitude","start station name" , "end station name","start station latitude","hour","end station latitude","end station longitude","start station id","end station id"]]
        hour_val = val[val["hour"]==hour]
        route(start_station,end_station,fig)
    return fig.add_child(plot_station(hour_val,tiles,val))
    

interact(f,start_station=start_stations,end_station=end_stations,hour=hour,tiles=map_options)

interactive(children=(Dropdown(description='start_station', options={'DeKalb Ave & Hudson Ave': 324, 'W 20 St & 8 Ave': 470, 'Greenwich St & W Houston St': 347, 'Stagg St & Union Ave': 3077, 'Carmine St & 6 Ave': 368, '5 Ave & E 63 St': 3136, 'W 70 St & Amsterdam Ave': 3175, 'Greenwich Ave & Charles St': 383, 'E 10 St & 5 Ave': 280, 'Madison St & Montgomery St': 291, 'Canal St & Rutgers St': 307, 'W 13 St & 6 Ave': 345, 'Frederick Douglass Blvd & W 117 St': 3534, 'W 31 St & 7 Ave': 379, 'Allen St & Stanton St': 312, 'Broadway & W 29 St': 486, 'W 52 St & 9 Ave': 449, 'Broadway & W 60 St': 499, 'E 9 St & Avenue C': 394, 'E 15 St & 3 Ave': 297, 'Kent Ave & N 7 St': 3016, 'E 58 St & 3 Ave': 305, '9 Ave & W 45 St': 479, 'Little West St & 1 Pl': 2008, '5 Ave & E 78 St': 3143, 'E 24 St & Park Ave S': 491, '31 St & Astoria Blvd': 3516, 'Central Park W & W 96 St': 3295, 'LaGuardia Pl & W 3 St': 161, 'St Marks Pl & 1 Ave': 438, 'W 18 St & 6 Ave': 168, 'W 22 St & 8 Ave': 453, 'Boerum St & Broadway': 3071, 'Christopher St & Greenwich St': 358, 'Cooper Square & Astor Pl': 3263, 'Adelphi St & Myrtle Ave': 270, 'St James Pl & Oliver St': 308, 'E 11 St & 1 Ave': 326, '6 Ave & 9 St': 3304, 'E 12 St & 3 Ave': 483, 'Forsyth St & Broome St': 531, 'E 85 St & 3 Ave': 3147, '1 Ave & E 18 St': 2003, 'Bushwick Ave & Powers St': 3083, 'E 25 St & 2 Ave': 507, 'MacDougal St & Prince St': 128, 'S 3 St & Bedford Ave': 481, 'W 24 St & 7 Ave': 446, 'Lafayette Ave & Fort Greene Pl': 274, 'W 33 St & 7 Ave': 492, '14 St & 5 Ave': 3319, 'Broadway & W 56 St': 468, 'Grand St & Havemeyer St': 471, 'Meserole Ave & Manhattan Ave': 3110, '31 St & Thomson Ave': 3123, 'Greenpoint Ave & Manhattan Ave': 3113, 'E 20 St & 2 Ave': 461, 'Great Jones St': 229, 'E 27 St & 1 Ave': 2012, 'Bank St & Hudson St': 346, '6 Ave & Spring St': 3474, 'Greenwich Ave & 8 Ave': 284, 'University Pl & E 14 St': 382, '11 Ave & W 59 St': 530, 'W 67 St & Broadway': 3159, 'W 100 St & Broadway': 3350, '11 Ave & W 41 St': 478, 'Barrow St & Hudson St': 127, 'St Marks Pl & 2 Ave': 236, 'E 10 St & Avenue A': 445, 'E 17 St & Broadway': 497, '45 Rd & 11 St': 3125, 'Broadway & W 41 St': 465, 'Metropolitan Ave & Bedford Ave': 539, 'Centre St & Chambers St': 387, 'W 34 St & 11 Ave': 525, '31 St & Hoyt Ave N': 3517, 'W 49 St & 8 Ave': 450, 'W 41 St & 8 Ave': 477, 'Columbia St & Kane St': 3311, 'Sullivan St & Washington Sq': 336, 'W 84 St & Broadway': 3177, 'Macon St & Nostrand Ave': 437, 'W 45 St & 6 Ave': 3466, 'Scholes St & Manhattan Ave': 3076, 'W 13 St & 5 Ave': 253, 'Schermerhorn St & 3 Ave': 3455, 'Barclay St & Church St': 417, 'Catherine St & Monroe St': 2009, 'Liberty St & Broadway': 195, 'E 11 St & 2 Ave': 237, '31 St & 23 Ave': 3526, 'Graham Ave & Grand St': 3081, 'Centre St & Worth St': 386, 'Frederick Douglass Blvd & W 112 St': 3553, 'Grand St & Elizabeth St': 3435, 'Harrison St & Hudson St': 249, 'Pacific St & Nevins St': 3412, 'Pershing Square North': 519, 'Amsterdam Ave & W 79 St': 3360, 'MacDougal St & Washington Sq': 252, '8 Ave & W 33 St': 490, 'Broadway & W 49 St': 173, '44 Dr & 21 St': 3615, 'E 31 St & 3 Ave': 476, 'Huron St & Franklin St': 3116, 'Bedford Ave & Nassau Ave': 3107, 'Broadway & E 14 St': 285, 'Lafayette Ave & St James Pl': 399, '2 Ave & E 31 St': 528, 'Central Park S & 6 Ave': 2006, 'Amsterdam Ave & W 73 St': 3167, '1 Ave & E 16 St': 504, 'Vernon Blvd & 50 Ave': 3119, 'Pike St & Monroe St': 331, 'Clinton Ave & Flushing Ave': 343, 'Mercer St & Bleecker St': 3260, 'E 51 St & 1 Ave': 454, 'W 27 St & 7 Ave': 442, 'Lawrence St & Willoughby St': 323, 'Fulton St & Rockwell Pl': 243, 'E 53 St & Madison Ave': 456, 'Montrose Ave & Bushwick Ave': 3074, 'W 4 St & 7 Ave S': 380, '3 Ave & E 62 St': 3134, 'Washington Pl & 6 Ave': 369, 'Myrtle Ave & Lewis Ave': 3064, 'Willoughby Ave & Hall St': 244, 'Fulton St & Broadway': 319, 'Rogers Ave & Sterling St': 3604, 'E 47 St & 1 Ave': 516, 'Central Park North & Adam Clayton Powell Blvd': 3374, 'Avenue D & E 8 St': 266, 'Court St & State St': 340

<function __main__.f>

In [16]:
data["speed"] = data["speed"].replace(0.000000,5.793497871920003)
data["speed"] = data["speed"].round()
data["speed"] = [int(x) for x in data["speed"]]
data["speed"].value_counts().reset_index()

,index,speed
0,6,229745
1,5,192427
2,7,160004
3,4,101657
4,8,83249
5,3,39423
6,9,34270
7,2,18454
8,1,11616
9,10,11497


In [17]:
speed = {"1": 1,"2": 2,"3": 3, "4": 4,"5": 5,"6": 6,"7": 7,"8": 8,"9": 9,"10":10, "11": 11,"12": 12,"13": 13,"14": 14,"15": 15,"16": 16,"41": 41}

In [18]:
data["day"].max()

31

In [19]:
data["dayOfWeek"].value_counts()

4    163082
1    148028
0    138115
2    136786
3    123874
6     95731
5     84351
Name: dayOfWeek, dtype: int64

In [20]:
dayofweek = {"Monday":0,"Tuesday":1,"Wednesday":2,"Thursday":3,"Friday":4,"Saturday":5,"Sunday":6}

In [21]:


def f(Speed,Days_of_Week):
    map_hooray = folium.Map(location=[40.738, -73.98],
                    zoom_start = 13) 

    # Ensure you're handing it floats
    data['start station latitude'] = data['start station latitude'].astype(float)
    data['start station longitude'] = data['start station longitude'].astype(float)

    # Filter the DF for rows, then columns, then remove NaNs
    heat_df = data[data['dayOfWeek']==Days_of_Week] # Reducing data size so it runs faster
    #heat_df = heat_df[data['speed']==Speed]
    heat_df = heat_df[heat_df['speed']==Speed] # Reducing data size so it runs faster
    #print(heat_df)
    heat_df = heat_df[['start station latitude', 'start station longitude']]
    heat_df = heat_df.dropna(axis=0, subset=['start station latitude','start station longitude'])
    #print(heat_df)
    # List comprehension to make out list of lists
    heat_data = [[row['start station latitude'],row['start station longitude']] for index, row in heat_df.iterrows()]
    #print(heat_data)
    # Plot it on the map
    if(len(heat_data)==0):
        print("No speed found!")
    HeatMap(heat_data).add_to(map_hooray)

# Display the map
    return map_hooray
interact(f,Speed=speed,Days_of_Week=dayofweek)

interactive(children=(Dropdown(description='Speed', options={'1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '10': 10, '11': 11, '12': 12, '13': 13, '14': 14, '15': 15, '16': 16, '41': 41}, value=1), Dropdown(description='Days_of_Week', options={'Monday': 0, 'Tuesday': 1, 'Wednesday': 2, 'Thursday': 3, 'Friday': 4, 'Saturday': 5, 'Sunday': 6}, value=0), Output()), _dom_classes=('widget-interact',))

<function __main__.f>

## chloropeth

In [22]:
location = data.groupby("start station id").first()
location["idval"] = location.index
trip = location.loc[:,["start station latitude","start station longitude","bikeid","idval","start station name"]]
val = data.groupby("start station id").count().iloc[:,[11]]
val.columns=["Count of bikes"]
trip = val.join(trip)

In [23]:
def df_to_geojson(trip, properties, lat='start station latitude', lon='start station longitude'):
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in trip.iterrows():
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}
        feature['geometry']['coordinates'] = [row[lon],row[lat]]
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson